In [ ]:
## 优先用ChatTongyi

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from pydantic import BaseModel, Field
from langchain_community.chat_models import ChatTongyi


In [2]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "build_effective_agents"

In [37]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_BASE_URL")
)

In [ ]:
# openai兼容模式qwen,不要用这种方式
llm = ChatOpenAI(
    model="qwen-max",
    base_url='https://dashscope.aliyuncs.com/compatible-mode/v1',
    api_key=os.getenv("DASHSCOPE_API_KEY")    
)

In [ ]:
tongyi_chat = ChatTongyi(
    model="qwen-max",
)

tongyi_chat.invoke("What is the capital of France?")

In [15]:
# 测试
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="J'aime la programmation.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 37, 'total_tokens': 44, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'qwen-plus', 'system_fingerprint': None, 'id': 'chatcmpl-7e14267c-ef72-98c3-b43d-4d547248a501', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--6f9e27e7-55aa-4884-84c4-f03ff8ee064f-0', usage_metadata={'input_tokens': 37, 'output_tokens': 7, 'total_tokens': 44, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

In [18]:
class GetWeather(BaseModel):
    """Get the current weather in a given location"""

    location: str = Field(..., description="The city and state, e.g. San Francisco, CA")

In [21]:
llm_with_tools = llm.bind_tools([GetWeather])

ai_msg = llm_with_tools.invoke(
    "what is the weather like in San Francisco",
)
ai_msg

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_lNk5cct9lltWV8wXvFYbdCRU', 'function': {'arguments': '{"location":"San Francisco, CA"}', 'name': 'GetWeather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 68, 'total_tokens': 86, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-BsQ2gc3OcJukKifQyH5nkNAVxOtCx', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--49de96aa-a497-4dba-8392-aa465abc2077-0', tool_calls=[{'name': 'GetWeather', 'args': {'location': 'San Francisco, CA'}, 'id': 'call_lNk5cct9lltWV8wXvFYbdCRU', 'type': 'tool_call'}], usage_metadata={'input_tokens': 68, 'output_tokens': 18, 'total_tokens': 86, 'input

In [22]:
def get_weather(location: str) -> None:
    """Get weather at a location."""
    return "It's sunny."


class OutputSchema(BaseModel):
    """Schema for response."""

    answer: str
    justification: str

In [29]:
llm.model_name

'qwen-plus'

In [33]:
structured_llm = llm.bind_tools(
    [get_weather],
)

In [34]:
tool_call_response = structured_llm.invoke("What is the weather in SF?")

In [35]:
tool_call_response.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'SF'},
  'id': 'call_0a7c205605654a7b9b74ad',
  'type': 'tool_call'}]

In [58]:
# Schema for structured output
from pydantic import BaseModel, Field

class SearchQuery(BaseModel):
    search_query: str = Field(None, description="Query that is optimized web search.")
    justification: str = Field(
        None, description="Why this query is relevant to the user's request."
    )


# Augment the LLM with schema for structured output
structured_llm = tongyi_chat.with_structured_output(SearchQuery)

# Invoke the augmented LLM
# 注意：千问模型在使用结构化输出时需要在消息中提到'json'
output = structured_llm.invoke(f"How does Calcium CT score relate to high cholesterol? Please provide the answer in following JSON format.\n{SearchQuery.model_json_schema()}")

In [59]:
output

SearchQuery(search_query='How does Calcium CT score relate to high cholesterol', justification='The user wants to know the relationship between Calcium CT score and high cholesterol. This search query will help in finding relevant information.')

In [50]:
llm.model_name

'qwen-max'

In [60]:
class GetWeather(BaseModel):
    '''Get the current weather in a given location'''

    location: str = Field(
        ..., description="The city and state, e.g. San Francisco, CA"
    )


class GetPopulation(BaseModel):
    '''Get the current population in a given location'''

    location: str = Field(
        ..., description="The city and state, e.g. San Francisco, CA"
    )

chat_with_tools = tongyi_chat.bind_tools([GetWeather, GetPopulation])
ai_msg = chat_with_tools.invoke(
    "Which city is hotter today and which is bigger: LA or NY?"
)
ai_msg.tool_calls

[{'name': 'GetWeather',
  'args': {'location': 'Los Angeles, CA'},
  'id': 'call_17e021f8060d45d8bc33ac',
  'type': 'tool_call'}]

In [61]:
ai_msg

AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'name': 'GetWeather', 'arguments': '{"location": "Los Angeles, CA"}'}, 'index': 0, 'id': 'call_17e021f8060d45d8bc33ac', 'type': 'function'}]}, response_metadata={'model_name': 'qwen-max', 'finish_reason': 'tool_calls', 'request_id': '78d482b2-fc9f-904e-a9ac-763e9f4ea449', 'token_usage': {'input_tokens': 265, 'output_tokens': 22, 'total_tokens': 287, 'prompt_tokens_details': {'cached_tokens': 0}}}, id='run--2cb2d471-faa3-486f-8a3f-057ea8addcbb-0', tool_calls=[{'name': 'GetWeather', 'args': {'location': 'Los Angeles, CA'}, 'id': 'call_17e021f8060d45d8bc33ac', 'type': 'tool_call'}])

In [62]:
from typing import Optional
class Joke(BaseModel):
    '''Joke to tell user.'''

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(description="How funny the joke is, from 1 to 10")


structured_chat = tongyi_chat.with_structured_output(Joke)
structured_chat.invoke("Tell me a joke about cats")

Joke(setup="Why don't cats play poker in the jungle?", punchline='Too many cheetahs!', rating=7)